In [1]:

import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm

from lvmsurveysim.utils.sqlite2astropy import peewee2astropy
from lvmdrp.core.image import Image, loadImage, combineImages
from lvmdrp.functions.imageMethod import preprocRawFrame_drp, basicCalibration_drp, createMasterFrame_drp

from lvmdrp.core.constants import CONFIG_PATH
from lvmdrp.main import load_master_config
from lvmdrp.utils.database import ReductionStatus, QualityFlag, create_or_connect_db, LVMFrames, get_raws_metadata


config = load_master_config()
db = create_or_connect_db(config)


In [2]:
lvm_frames = peewee2astropy(LVMFrames).to_pandas().set_index("id")
lvm_frames.dropna(subset=("mjd", "spec", "ccd", "exptime", "imagetyp", "obstime"), how="any", axis="index", inplace=True)

lvm_frames.datetime = lvm_frames.datetime.apply(pd.to_datetime)
lvm_frames.obstime = lvm_frames.obstime.apply(pd.to_datetime)
lvm_frames

,datetime,mjd,spec,ccd,exptime,imagetyp,obstime,observat,label,path,...,ffs,mi150,ts,ldls,nirled,reduction_started,reduction_finished,status,flags,calib_id
id,,,,,,,,,,,,,,,,,,,,,
1,2023-02-13 16:56:44.928418,59786,sp3,z3,5.0,object,2022-07-26 03:54:05.955,SBS,sdR-s-z3-00005247,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
2,2023-02-13 16:56:44.928513,59786,sp3,r3,6.0,object,2022-07-26 03:32:28.081,SBS,sdR-s-r3-00005245,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
3,2023-02-13 16:56:44.928586,59786,sp3,r3,0.0,bias,2022-07-26 03:39:38.822,SBS,sdR-s-r3-00005246,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
4,2023-02-13 16:56:44.928656,59786,sp3,z3,6.0,object,2022-07-26 03:32:28.081,SBS,sdR-s-z3-00005245,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
5,2023-02-13 16:56:44.928724,59786,sp3,r3,5.0,object,2022-07-26 03:54:05.955,SBS,sdR-s-r3-00005247,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19724,2023-02-13 16:56:49.886670,59678,sp1,b1,300.0,continuum,2022-04-09 01:04:27.451,LCO,sdR-s-b1-00003273,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,1,0,1,0,None,None,1,1,None
19725,2023-02-13 16:56:49.886738,59678,sp1,r1,120.0,continuum,2022-04-09 01:27:26.891,LCO,sdR-s-r1-00003274,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,1,0,1,0,None,None,1,1,None
19726,2023-02-13 16:56:49.886837,59678,sp1,z1,2.0,continuum,2022-04-09 00:53:19.274,LCO,sdR-s-z1-00003272,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,1,0,1,0,None,None,1,1,None


In [3]:
# filter recent frames
# raw_frames = lvm_frames.loc[lvm_frames.obstime >= datetime.now() - timedelta(days=240)]
# filter spec3 frames
raw_frames = lvm_frames.loc[lvm_frames.ccd.str.endswith("3")]
# filter calibration frames
cal_frames = raw_frames.loc[raw_frames.imagetyp.isin(["bias", "dark", "flat"])].sort_values("imagetyp", key=lambda x: x.map({"bias":0, "dark":1, "flat":2}))
sci_frames = raw_frames.loc[raw_frames.imagetyp == "object"]

gr = cal_frames.groupby(["imagetyp", "ccd", "exptime"])
# gr.groups

In [4]:
cal_path = os.path.join("_data-lvm", "calibration")
sci_path = os.path.join("_data-lvm", "science")

os.makedirs(cal_path, exist_ok=True)
os.makedirs(sci_path, exist_ok=True)

# preprocess individual frames
for idx, cal_frame in tqdm(cal_frames.iterrows(), total=len(cal_frames), desc="reducing calibration frames", unit="frame", ascii=True):
    out_cal_path = os.path.join(cal_path, f"{cal_frame.label}.pre.fits")
    # skip done frames
    if os.path.isfile(out_cal_path): continue
    preprocRawFrame_drp(
        in_image=cal_frame.path,
        out_image=out_cal_path,
        compute_error=False,
        unit="e-"
    )

for idx, sci_frame in tqdm(sci_frames.iterrows(), total=len(sci_frames), desc="reducing science frames", unit="frame", ascii=True):
    out_sci_path = os.path.join(sci_path, f"{sci_frame.label}.pre.fits")
    # skip done frames
    if os.path.isfile(out_sci_path): continue
    preprocRawFrame_drp(
        in_image=sci_frame.path,
        out_image=out_sci_path,
        compute_error=True,
        unit="e-"
    )

reducing calibration frames:  71%|#######1  | 1240/1740 [17:52<07:12,  1.16frame/s]


OSError: Not enough space on disk: requested 66585600, available 0. Not enough space on disk: requested 66585600, available 0. Not enough free space to write 66585600 bytes

In [ ]:

# combine calibration frames into masters
for cal_group, idx in gr.groups.items():
    pass



In [ ]:

# apply calibration frames to object frames

